# WGAN and WGAN-GP

## All Required Libs and Parameters

In [1]:
# All required libs are imporeted in this cell
import os, glob, multiprocessing
import tensorflow as tf
import  numpy as np
from PIL import Image
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Reshape, Conv2DTranspose, ReLU, LeakyReLU, Activation
from tensorflow.keras import Sequential, optimizers, Input, layers
from tensorflow import keras
print(tf.__version__)

z_dim = 100
epochs = 3000000
batch_size = 128
learning_rate = 0.0002
is_training = True

2.2.0-dev20200315


## All Required Functions/Methods

In [6]:
def make_anime_dataset(img_paths, batch_size, resize=64, drop_remainder=True, shuffle=True, repeat=1):

    # @tf.function
    def _map_fn(img):
        img = tf.image.resize(img, [resize, resize])
        # img = tf.image.random_crop(img,[resize, resize])
        # img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.clip_by_value(img, 0, 255)
        img = img / 127.5 - 1 #-1~1
        return img

    dataset = disk_image_batch_dataset(img_paths,
                                          batch_size,
                                          drop_remainder=drop_remainder,
                                          map_fn=_map_fn,
                                          shuffle=shuffle,
                                          repeat=repeat)
    img_shape = (resize, resize, 3)
    len_dataset = len(img_paths) // batch_size

    return dataset, img_shape, len_dataset


def batch_dataset(dataset,
                  batch_size,
                  drop_remainder=True,
                  n_prefetch_batch=1,
                  filter_fn=None,
                  map_fn=None,
                  n_map_threads=None,
                  filter_after_map=False,
                  shuffle=True,
                  shuffle_buffer_size=None,
                  repeat=None):
    # set defaults
    if n_map_threads is None:
        n_map_threads = multiprocessing.cpu_count()
    if shuffle and shuffle_buffer_size is None:
        shuffle_buffer_size = max(batch_size * 128, 2048)  # set the minimum buffer size as 2048

    # [*] it is efficient to conduct `shuffle` before `map`/`filter` because `map`/`filter` is sometimes costly
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size)

    if not filter_after_map:
        if filter_fn:
            dataset = dataset.filter(filter_fn)

        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

    else:  # [*] this is slower
        if map_fn:
            dataset = dataset.map(map_fn, num_parallel_calls=n_map_threads)

        if filter_fn:
            dataset = dataset.filter(filter_fn)

    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    dataset = dataset.repeat(repeat).prefetch(n_prefetch_batch)

    return dataset


def memory_data_batch_dataset(memory_data,
                              batch_size,
                              drop_remainder=True,
                              n_prefetch_batch=1,
                              filter_fn=None,
                              map_fn=None,
                              n_map_threads=None,
                              filter_after_map=False,
                              shuffle=True,
                              shuffle_buffer_size=None,
                              repeat=None):
    """Batch dataset of memory data.
    Parameters
    ----------
    memory_data : nested structure of tensors/ndarrays/lists
    """
    dataset = tf.data.Dataset.from_tensor_slices(memory_data)
    dataset = batch_dataset(dataset,
                            batch_size,
                            drop_remainder=drop_remainder,
                            n_prefetch_batch=n_prefetch_batch,
                            filter_fn=filter_fn,
                            map_fn=map_fn,
                            n_map_threads=n_map_threads,
                            filter_after_map=filter_after_map,
                            shuffle=shuffle,
                            shuffle_buffer_size=shuffle_buffer_size,
                            repeat=repeat)
    return dataset


def disk_image_batch_dataset(img_paths,
                             batch_size,
                             labels=None,
                             drop_remainder=True,
                             n_prefetch_batch=1,
                             filter_fn=None,
                             map_fn=None,
                             n_map_threads=None,
                             filter_after_map=False,
                             shuffle=True,
                             shuffle_buffer_size=None,
                             repeat=None):
    """Batch dataset of disk image for PNG and JPEG.
    Parameters
    ----------
        img_paths : 1d-tensor/ndarray/list of str
        labels : nested structure of tensors/ndarrays/lists
    """
    if labels is None:
        memory_data = img_paths
    else:
        memory_data = (img_paths, labels)

    def parse_fn(path, *label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)  # fix channels to 3
        return (img,) + label

    if map_fn:  # fuse `map_fn` and `parse_fn`
        def map_fn_(*args):
            return map_fn(*parse_fn(*args))
    else:
        map_fn_ = parse_fn

    dataset = memory_data_batch_dataset(memory_data,
                                        batch_size,
                                        drop_remainder=drop_remainder,
                                        n_prefetch_batch=n_prefetch_batch,
                                        filter_fn=filter_fn,
                                        map_fn=map_fn_,
                                        n_map_threads=n_map_threads,
                                        filter_after_map=filter_after_map,
                                        shuffle=shuffle,
                                        shuffle_buffer_size=shuffle_buffer_size,
                                        repeat=repeat)

    return dataset

def get_random_z(z_dim, batch_size):
    return tf.random.uniform([batch_size, z_dim], minval=-1, maxval=1)


 
def save_result(val_out, val_block_size, image_path, color_mode):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        # img = img.astype(np.uint8)
        return img

    preprocesed = preprocess(val_out)
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    #toimage(final_image).save(image_path)
    #print(image_path)
    Image.fromarray(final_image).save(image_path)
img_path = glob.glob('./faces/*.jpg')
print("Num of Files:",len(img_path))

dataset, img_shape, _ = make_anime_dataset(img_path, batch_size, resize = 64)
print(dataset, img_shape)
sample = next(iter(dataset))
print(sample.shape, tf.reduce_max(sample).numpy(), tf.reduce_min(sample).numpy())
dataset = dataset.repeat(100)
db_iter = iter(dataset)

Num of Files: 51223
<PrefetchDataset shapes: (128, 64, 64, 3), types: tf.float32> (64, 64, 3)
(128, 64, 64, 3) 1.0 -1.0


## Model Definitions and Test Code
### SubClassing Mode

In [3]:
# WGAN can re-use DCGAN with minor modifications:
# No BatchNormalization, No sigmoid output for discriminator
class Generator(keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.s = 2
        self.k = 4
        self.n_f = 1024
        
        self.dense1 = Dense(self.s * self.s * self.n_f)
        self.reshape1 = Reshape(target_shape = (self.s, self.s, self.n_f))
        self.conv1 = Conv2DTranspose(512, self.k, 2, 'same')
        self.conv2 = Conv2DTranspose(256, self.k, 2, 'same')
        self.conv3 = Conv2DTranspose(128, self.k, 2, 'same')
        self.conv4 = Conv2DTranspose(64, self.k, 2, 'same')
        self.conv5 = Conv2DTranspose(3, self.k, 2, 'same')
        
    def call(self, inputs, training=None):
        x = inputs
        x = self.dense1(x)
        x = tf.nn.leaky_relu(x)
        x = self.reshape1(x)
        
        x = tf.nn.leaky_relu(self.conv1(x))
        x = tf.nn.leaky_relu(self.conv2(x))
        x = tf.nn.leaky_relu(self.conv3(x))
        x = tf.nn.leaky_relu(self.conv4(x))
        x = tf.tanh(self.conv5(x))
        
        return x
class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.s = 4
        self.k = 4
        
        self.conv1 = Conv2D(64, self.k, 2, 'same')
        self.conv2 = Conv2D(128, self.k, 2, 'same')
        self.conv3 = Conv2D(256, self.k, 2, 'same')
        self.conv4 = Conv2D(512, self.k, 2, 'same')
        self.flatten = Flatten()
        self.dense = Dense(1)
    
    def call(self, inputs, training = None):
        x = inputs
        x = tf.nn.leaky_relu(self.conv1(x))
        x = tf.nn.leaky_relu(self.conv2(x))
        x = tf.nn.leaky_relu(self.conv3(x))
        x = tf.nn.leaky_relu(self.conv4(x))
        x = self.dense(self.flatten(x))
        
        return x
    
# Generator_s and Discriminator_s are lite versions of DCGAN
class Generator_s(keras.Model):

    def __init__(self):
        super(Generator_s, self).__init__()

        # z: [b, 100] => [b, 3*3*512] => [b, 3, 3, 512] => [b, 64, 64, 3]
        self.fc = layers.Dense(3*3*512)

        self.conv1 = layers.Conv2DTranspose(256, 3, 3, 'valid')
        self.bn1 = layers.BatchNormalization()

        self.conv2 = layers.Conv2DTranspose(128, 5, 2, 'valid')
        self.bn2 = layers.BatchNormalization()

        self.conv3 = layers.Conv2DTranspose(3, 4, 3, 'valid')

    def call(self, inputs, training=None):
        # [z, 100] => [z, 3*3*512]
        x = self.fc(inputs)
        x = tf.reshape(x, [-1, 3, 3, 512])
        x = tf.nn.leaky_relu(x)

        #
        x = tf.nn.leaky_relu(self.bn1(self.conv1(x), training=training))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
        x = self.conv3(x)
        x = tf.tanh(x)

        return x


class Discriminator_s(keras.Model):

    def __init__(self):
        super(Discriminator_s, self).__init__()

        # [b, 64, 64, 3] => [b, 1]
        self.conv1 = layers.Conv2D(64, 5, 3, 'valid')

        self.conv2 = layers.Conv2D(128, 5, 3, 'valid')
        self.bn2 = layers.BatchNormalization()

        self.conv3 = layers.Conv2D(256, 5, 3, 'valid')
        self.bn3 = layers.BatchNormalization()

        # [b, h, w ,c] => [b, -1]
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(1)


    def call(self, inputs, training=None):

        x = tf.nn.leaky_relu(self.conv1(inputs))
        x = tf.nn.leaky_relu(self.bn2(self.conv2(x), training=training))
        x = tf.nn.leaky_relu(self.bn3(self.conv3(x), training=training))

        # [b, h, w, c] => [b, -1]
        x = self.flatten(x)
        # [b, -1] => [b, 1]
        logits = self.fc(x)

        return logits

def gradient_penalty(discriminator, batch_x, fake_image):

    batchsz = batch_x.shape[0]

    # [b, h, w, c]
    t = tf.random.uniform([batchsz, 1, 1, 1])
    # [b, 1, 1, 1] => [b, h, w, c]
    t = tf.broadcast_to(t, batch_x.shape)

    interplate = t * batch_x + (1 - t) * fake_image

    with tf.GradientTape() as tape:
        tape.watch([interplate])
        d_interplote_logits = discriminator(interplate, training=True)
    grads = tape.gradient(d_interplote_logits, interplate)

    # grads:[b, h, w, c] => [b, -1]
    grads = tf.reshape(grads, [grads.shape[0], -1])
    gp = tf.norm(grads, axis=1) #[b]
    gp = tf.reduce_mean( (gp-1)**2 )

    return gp

# WGAN has different loss functions for generator and discriminator. 
def d_loss_fn(generator, discriminator, batch_z, batch_x, is_training):
    # 1. treat real image as real
    # 2. treat generated image as fake
    fake_image = generator(batch_z, is_training)
    d_fake_logits = discriminator(fake_image, is_training)
    d_real_logits = discriminator(batch_x, is_training)
    loss = tf.reduce_mean(d_fake_logits) - tf.reduce_mean(d_real_logits)
    return loss


def g_loss_fn(generator, discriminator, batch_z, is_training):
    fake_image = generator(batch_z, is_training)
    d_fake_logits = discriminator(fake_image, is_training)
    loss = -tf.reduce_mean(d_fake_logits)
    return loss



In [ ]:
# WGAN
def main():
    
    with tf.device('/GPU:0'):
        generator = Generator_s()
        generator.build(input_shape = (None, z_dim))
        # generator.summary()

        discriminator = Discriminator_s()
        discriminator.build(input_shape = (None, 64, 64, 3))
        # discriminator.summary()

    # Not to use momentum-based optimizers for WGAN, such as Adam. In contrast, use RMSProp or SGD
        # g_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
        # d_opt = optimizers.Adam(learning_rate = learning_rate, beta_1=0.5)
        g_opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        d_opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        #g_opt = optimizers.RMSProp

        d_losses, g_losses = [], []

        for epoch in range(epochs):
            for _ in range(5):
                batch_z = get_random_z(z_dim, batch_size)
                batch_x = next(db_iter)

                with tf.GradientTape() as d_tape:
                    #fake_images = generator(batch_z, training = True)
                    #fake_outputs = discriminator(fake_images, training = True)
                    #real_outputs = discriminator(batch_x, training = True)
                    d_loss = d_loss_fn(generator, discriminator, batch_z, batch_x, is_training)
                    #d_loss = tf.reduce_mean(fake_outputs) - tf.reduce_mean(real_outputs)

                # One important difference between DCGAN and WGAN is the clipping process
                # that clip the weights of Discriminator into a range (e.g. -0.01 - 0.01)
                # every iteration.
                [p.assign(tf.clip_by_value(p,-0.01,0.01)) for p in discriminator.trainable_variables]
                d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
                d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))

                batch_zz = get_random_z(z_dim, batch_size)

                with tf.GradientTape() as g_tape:
                    #fake_outputs = discriminator(generator(batch_zz, training=True), training=True)
                    #g_loss = -tf.reduce_mean(fake_outputs)
                    g_loss = g_loss_fn(generator, discriminator, batch_z, is_training)

                g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
                g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))

            if epoch %100 ==0:
                print("Epoch:", epoch, "D-loss:", float(d_loss), "G-loss:", float(g_loss))            
                z = tf.random.uniform([100, z_dim], minval=-1, maxval=1)
                generated_images = generator(z, training = False)
                img_path = os.path.join('gan_images', 'wgan-dcgan-%d.png'%epoch)
                save_result(generated_images.numpy(), 10, img_path, color_mode='P')

                d_losses.append(float(d_loss))
                g_losses.append(float(g_loss))

            if epoch % 10000 == 1:
                generator.save_weights('generator.ckpt')
                discriminator.save_weights('discriminator.ckpt')

if __name__ == '__main__':
    main()
        

In [12]:
# WGAN-GP (Gradient Penalty)
# GP aims to constrain the weights of Discriminator to be close to 1
# The most parts of WGAN-GP are similar with WGAN.
# Differences: No clipping, but a penalty term in D's loss

def gradient_penalty(discriminator, real, fake):
    batch_sz = real.shape[0]
    t = tf.random.uniform([batch_sz, 1, 1, 1])
    t = tf.broadcast_to(t, real.shape)
    
    interplate = t * real + (1 - t) * fake
    
    with tf.GradientTape() as tape:
        tape.watch([interplate])
        d_interplate_logits = discriminator(interplate, True)
    grads = tape.gradient(d_interplate_logits, interplate)
    
    grads = tf.reshape(grads, [grads.shape[0], -1])
    gp = tf.norm(grads, axis = 1)
    gp = tf.reduce_mean((gp - 1.)**2)
    
    return gp

def gradient_penalty2(discriminator, batch_x, fake_image):

    batchsz = batch_x.shape[0]

    # [b, h, w, c]
    t = tf.random.uniform([batchsz, 1, 1, 1])
    # [b, 1, 1, 1] => [b, h, w, c]
    t = tf.broadcast_to(t, batch_x.shape)

    interplate = t * batch_x + (1 - t) * fake_image

    with tf.GradientTape() as tape:
        tape.watch([interplate])
        d_interplote_logits = discriminator(interplate, training=True)
    grads = tape.gradient(d_interplote_logits, interplate)

    # grads:[b, h, w, c] => [b, -1]
    grads = tf.reshape(grads, [grads.shape[0], -1])
    gp = tf.norm(grads, axis=1) #[b]
    gp = tf.reduce_mean( (gp-1)**2 )
    
    print(gp)

    return gp

# Override d_loss_fn(.)
def d_loss_gp_fn(g, d, z, real, l, is_training):
    fake_images = g(z, is_training)
    fake_outputs = d(fake_images, is_training)
    real_outputs = d(real, is_training)
    
    gp = gradient_penalty(d, real, fake_images)
    
    loss = tf.reduce_mean(fake_outputs) - tf.reduce_mean(real_outputs) + 10. * gp
    
    return loss, gp
    
import tensorflow as tf
def main():
    l = 10
    generator = Generator_s()
    generator.build(input_shape=(None, z_dim))

    discriminator = Discriminator_s()
    discriminator.build(input_shape=(None, 64, 64, 3))

    g_opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    d_opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

#    for epoch in range(epochs):
#        for _ in range(5):
#            batch_z = get_random_z(z_dim, batch_size)
#            batch_x = next(db_iter)
#
#            with tf.GradientTape() as d_tape:
#                d_loss, gp = d_loss_gp_fn(generator, discriminator, batch_z, batch_x, l, is_training)
#            d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
#            d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))
#
#        batch_zz = get_random_z(z_dim, batch_size)
#        with tf.GradientTape() as g_tape:
#            g_loss = g_loss_fn(generator, discriminator, batch_zz, is_training)
#        g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
#        g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))
    for epoch in range(epochs):
        batch_z = get_random_z(z_dim, batch_size)
        batch_x = next(db_iter)
        
        with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:
            d_loss, gp = d_loss_gp_fn(generator, discriminator, batch_z, batch_x, l, is_training)
            g_loss = g_loss_fn(generator, discriminator, batch_z, is_training)
        d_grads = d_tape.gradient(d_loss, discriminator.trainable_variables)
        g_grads = g_tape.gradient(g_loss, generator.trainable_variables)
        d_opt.apply_gradients(zip(d_grads, discriminator.trainable_variables))
        g_opt.apply_gradients(zip(g_grads, generator.trainable_variables))

        if epoch % 100 == 0:
            print(epoch, 'd-loss:',float(d_loss), 'g-loss:', float(g_loss), 'gp:', float(gp))
            z = tf.random.normal([100, z_dim])
            fake_image = generator(z, training=False)
            img_path = os.path.join('gan_images', 'dc-wgan-gp-%d.png'%epoch)
            save_result(fake_image.numpy(), 10, img_path, color_mode='P')
        
if __name__ == "__main__":
    main()

0 d-loss: 1.1007704734802246 g-loss: -0.4632747173309326 gp: 0.10838565975427628
100 d-loss: -0.7238081693649292 g-loss: -0.6365422010421753 gp: 0.021410498768091202
200 d-loss: -0.5758966207504272 g-loss: -1.0889592170715332 gp: 0.0505334734916687
300 d-loss: -0.0852719247341156 g-loss: -0.9664949178695679 gp: 0.026621194556355476
400 d-loss: -0.2409249246120453 g-loss: -1.0052525997161865 gp: 0.01939362660050392
500 d-loss: -0.06857191026210785 g-loss: -0.8566693663597107 gp: 0.019396914169192314
600 d-loss: -0.06188768148422241 g-loss: -0.7952290773391724 gp: 0.021045386791229248
700 d-loss: -0.2486681193113327 g-loss: -0.7828883528709412 gp: 0.009191693738102913
800 d-loss: -0.11653535068035126 g-loss: -0.8499849438667297 gp: 0.011796565726399422
900 d-loss: -0.2219536006450653 g-loss: -0.837039053440094 gp: 0.005095598753541708
1000 d-loss: -0.17695215344429016 g-loss: -0.802977442741394 gp: 0.006519088987261057
1100 d-loss: -0.09535479545593262 g-loss: -0.9423953890800476 gp: 0.0

9600 d-loss: -1.0403869152069092 g-loss: -1.953627109527588 gp: 0.008685003966093063
9700 d-loss: -1.036900520324707 g-loss: -1.6760950088500977 gp: 0.006835062988102436
9800 d-loss: -0.6649479866027832 g-loss: -1.3436706066131592 gp: 0.012149062007665634
9900 d-loss: -0.6027340888977051 g-loss: -2.3557980060577393 gp: 0.012798193842172623
10000 d-loss: -1.176830768585205 g-loss: -1.5072407722473145 gp: 0.007564421743154526
10100 d-loss: -1.2576117515563965 g-loss: -2.0789952278137207 gp: 0.006364081054925919
10200 d-loss: -0.7990856766700745 g-loss: -2.586362600326538 gp: 0.018680088222026825
10300 d-loss: -0.7625638246536255 g-loss: -1.738723635673523 gp: 0.012685154564678669
10400 d-loss: -0.8597558736801147 g-loss: -2.8692307472229004 gp: 0.009707987308502197
10500 d-loss: -1.352446436882019 g-loss: -1.60464346408844 gp: 0.009247985668480396
10600 d-loss: -0.9403713345527649 g-loss: -2.4664418697357178 gp: 0.01044723391532898
10700 d-loss: -1.0330400466918945 g-loss: -2.89632010459

19200 d-loss: -0.957658052444458 g-loss: -2.8267526626586914 gp: 0.01817009411752224
19300 d-loss: -1.035115122795105 g-loss: -3.991234064102173 gp: 0.012524373829364777
19400 d-loss: -1.0102733373641968 g-loss: -3.2674243450164795 gp: 0.009046114049851894
19500 d-loss: -1.354128122329712 g-loss: -3.1218760013580322 gp: 0.012844866141676903
19600 d-loss: -0.9791883230209351 g-loss: -2.8349249362945557 gp: 0.012695420533418655
19700 d-loss: -1.357460379600525 g-loss: -2.9970717430114746 gp: 0.016504153609275818
19800 d-loss: -0.8513644337654114 g-loss: -3.7471365928649902 gp: 0.008139201439917088
19900 d-loss: -1.0996615886688232 g-loss: -3.3166122436523438 gp: 0.01146185677498579
20000 d-loss: -0.8975555300712585 g-loss: -3.582353353500366 gp: 0.006888802163302898
20100 d-loss: -1.0538487434387207 g-loss: -3.7024006843566895 gp: 0.009463215246796608
20200 d-loss: -1.2296863794326782 g-loss: -3.0002927780151367 gp: 0.010037627071142197
20300 d-loss: -1.3445062637329102 g-loss: -4.073536

28800 d-loss: -1.7908847332000732 g-loss: -4.7696452140808105 gp: 0.01963537000119686
28900 d-loss: -0.7584096193313599 g-loss: -4.5177001953125 gp: 0.008932098746299744
29000 d-loss: -1.4666061401367188 g-loss: -3.5395491123199463 gp: 0.012038704939186573
29100 d-loss: -1.4551613330841064 g-loss: -4.2889604568481445 gp: 0.009584502317011356
29200 d-loss: -1.2123228311538696 g-loss: -3.476316213607788 gp: 0.006704746745526791
29300 d-loss: -1.6850897073745728 g-loss: -4.418304920196533 gp: 0.009497438557446003
29400 d-loss: -0.6597468256950378 g-loss: -3.683561086654663 gp: 0.010870367288589478
29500 d-loss: -1.0935466289520264 g-loss: -3.734968662261963 gp: 0.013593361712992191
29600 d-loss: -1.1596962213516235 g-loss: -3.7583751678466797 gp: 0.00879429467022419
29700 d-loss: -1.3059370517730713 g-loss: -4.289397716522217 gp: 0.011777287349104881
29800 d-loss: -1.082399845123291 g-loss: -3.729719877243042 gp: 0.015549378469586372
29900 d-loss: -1.2019621133804321 g-loss: -4.9952182769

38400 d-loss: -1.7216334342956543 g-loss: -4.367036819458008 gp: 0.012901635840535164
38500 d-loss: -1.2504725456237793 g-loss: -5.386634349822998 gp: 0.014136750251054764
38600 d-loss: -1.3280891180038452 g-loss: -6.0685625076293945 gp: 0.018591077998280525
38700 d-loss: -0.8464654684066772 g-loss: -5.051019668579102 gp: 0.009284893050789833
38800 d-loss: -1.0385921001434326 g-loss: -5.786165237426758 gp: 0.0309922955930233
38900 d-loss: -1.5101544857025146 g-loss: -5.344005584716797 gp: 0.01309186965227127
39000 d-loss: -1.318481683731079 g-loss: -5.240776062011719 gp: 0.025398042052984238
39100 d-loss: -1.44381844997406 g-loss: -4.489887237548828 gp: 0.011832917109131813
39200 d-loss: -1.4904979467391968 g-loss: -5.539308547973633 gp: 0.009813657961785793
39300 d-loss: -1.3814717531204224 g-loss: -5.506291389465332 gp: 0.011600198224186897
39400 d-loss: -1.8908910751342773 g-loss: -5.237240314483643 gp: 0.017740633338689804
39500 d-loss: -1.2666360139846802 g-loss: -4.43396759033203

StopIteration: 